In [10]:
import import_ipynb
import pandas as pd
import utils
import tensorflow as tf
from tensorflow import keras
from utils import CosineAnnealingLearningRateSchedule,set_log_checkpoint,plot_metrics,combined_loss,dice_coef
from config import IMG_WIDTH,IMG_HEIGHT,N_CHANNELS,BATCH_SIZE,N_CLASS,NB_EPOCH 
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
from sklearn.model_selection import train_test_split
img_size = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE
def load_data(data):
    '''
        Extracts Image and mask paths from input DF
        Args:
            data : DataFrame contains paths
        Returns:
                images : X-Ray image paths 
                masks :  X-Ray mask paths
    '''
    images = data['ImagePath']
    masks = data['MaskPath']
    return images, masks


def read_image(datapoint):
    '''
        Reads images that are stored into disk
        Args:
            datapoint : Image path
        Returns:
                image tensor 
    '''
    img = tf.io.read_file(datapoint)
    img = tf.image.decode_png(img, channels= N_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32) 
    image = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 

    image = exposure.equalize_adapthist(image)     # contrast correction
    return image

def read_mask(datapoint):
    '''
        Reads masks that are stored into disk
        Args:
            datapoint : Mask path
        Returns:
                Mask tensor 
    '''
    label = tf.io.read_file(datapoint)
    label = tf.image.decode_png(label, channels= 1)
    label = tf.image.convert_image_dtype(label, tf.float32) 
    mask = tf.image.resize(label, [IMG_WIDTH, IMG_HEIGHT])
                           
    return mask
    
def preprocess_train(image, image_mask):
    '''
        Reads images and masks that are stored into disk and applies appropriate tranformation
        Args:
            image : Image Tensor
            image_mask : mask Tensor
        Returns:
                Transformed Images and Masks 
    '''
    def f(image, image_mask):

        image = image.decode()
        image_mask = image_mask.decode()

        image = read_image(image)
        image_mask = read_mask(image_mask)
        
        a = tf.random.uniform((),minval=0,maxval=1)
        if a<0.2:
            image=tf.image.flip_left_right(image)
            image_mask=tf.image.flip_left_right(image_mask)
        if a<0.4 and a>0.2:
            image = tf.image.random_brightness(image, max_delta=0.15) # Random brightness
        if a<0.6 and a>0.4:
            image=tf.image.adjust_gamma(image, gamma=tf.random.uniform((),minval=0,maxval=1), gain=1)
        if a<0.8 and a>0.6:
            image=tf.image.random_contrast(image,lower=0.2,upper=0.3)
        if a<1.0 and a>0.8:
            image=tf.image.random_saturation(image, lower=2, upper=5)
            
        return image, image_mask
    images, masks = tf.numpy_function(f, [image, image_mask], [tf.float32, tf.float32])
    images.set_shape([img_size, img_size, 3])
    masks.set_shape([img_size, img_size, 1])

    return images, masks

def preprocess_test(image, image_mask):
    def f(image, image_mask):

        image = image.decode()
        image_mask = image_mask.decode()

        image = read_image(image)
        image_mask = read_mask(image_mask)

        return image, image_mask
    images, masks = tf.numpy_function(f, [image, image_mask], [tf.float32, tf.float32])
    images.set_shape([img_size, img_size, 3])
    masks.set_shape([img_size, img_size, 1])

    return images, masks

def tf_dataset(x, y, batch=8,type_ = 'train'):
    '''
        Creates train and test data pipeline that will be used while model training 
        Args:
            x : Image paths
            y : Mask paths
            batch : batch size to be used
            type_ : type of data train or test
        Returns:
                dataset object 
            
    '''
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.cache().shuffle(buffer_size=15000)
    if type_ == 'train':
        dataset = dataset.map(preprocess_train,num_parallel_calls=AUTOTUNE)
    else:
        dataset = dataset.map(preprocess_test,num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


def get_segmentaion_model(name= 'Uefficientnetb4', BACKBONE = 'efficientnetb4',ENCODER_WEIGHTS = 'imagenet'):
    '''
        Creates segmentaion model object and compiles it with Adam optimizer and combined_loss function
        Args:
            name : Name of model 
            BACKBONE : BACKBONE model name to be used as encider part
            ENCODER_WEIGHTS : weights with which to assign the encoder
        Returns:
                complied segmentaion model object 
            
    '''
    tf.random.set_seed(100) # Set global seed
    keras.backend.clear_session()  # For easy reset of notebook state.

    model_seg = sm.Unet(BACKBONE,  input_shape=(256, 256, 3), encoder_weights= ENCODER_WEIGHTS)
    #keras.utils.plot_model(model_seg, name + ".png", show_shapes=True)

    model_seg.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss=combined_loss , metrics=[dice_coef])

    return model_seg

def train_and_plot_metrics(model,name,cp_callback,tensorboard_callback,train_dataset,test_dataset):
    '''
        Trains model on train data and plots different training metrics
        Args:
            model : Model object to which to feed the data 
            cp_callback : Model checkpoint callback object
            tensorboard_callback : tensorboard callback obejct 
            train_dataset : train_dataset obejct
            test_dataset: test_dataset obejct 
        Returns:
                None
            
    '''
    history = model.fit(train_dataset,                           
                            use_multiprocessing=True,
                            epochs=NB_EPOCH,
                            batch_size = BATCH_SIZE,
                            steps_per_epoch = len(train_dataset),
                            validation_data= test_dataset,
                            verbose=1,
                            callbacks=[cp_callback, tensorboard_callback ]
                        )
    
    # Loading best saved model
    model.load_weights(CHECKPOINT_PATH)
    # saving training history 
    np.save('./training/' + name + '.npy',history.history)
    #saving model with architecture
    model.save('./models/' + name)
    plot_metrics(history)


def train_model():
    df_meta_final = pd.read_pickle('./df_meta_final.pkl')
    df_meta_final['class_'] = df_meta_final['class_'].map({'Pneumothorax' :1 ,'NotPneumothorax' : 0})
    X_train, X_test, y_train, y_test = train_test_split(df_meta_final, df_meta_final['class_'],stratify =df_meta_final['class_'],  test_size=0.20, random_state=42)
    
    images, masks = load_data(X_train)
    train_dataset = tf_dataset(images, masks,BATCH_SIZE )

    images, masks = load_data(X_test)
    test_dataset = tf_dataset(images, masks, BATCH_SIZE,type_='test')
    
    name= 'Uefficientnetb4' 
    model_seg = get_segmentaion_model(name, BACKBONE = 'efficientnetb4')
    cp_callback,tensorboard_callback,ca = set_log_checkpoint(name)
    train_and_plot_metrics(model_seg,name,cp_callback,tensorboard_callback,train_dataset,test_dataset)

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from config.ipynb
env: SM_FRAMEWORK=tf.keras


Using TensorFlow backend.


Segmentation Models: using `tf.keras` framework.


In [3]:
# train_model()